In [2]:
import requests
import time
import json
import numpy as np
import blockchain

In [3]:
url = "https://api.thingspeak.com/channels/1704499/feeds.json?api_key=Z70IVJ1X27AVPQ6C&results=2"
Rkey = "MC9FFB16PZV2BSWR"
R = 6373.0 # world radius
latStation = [7.865661,7.881308,7.893300,7.887864,7.890755,7.882510,7.885414] # lat of stantion point
lonstation = [98.397750,98.364982,98.368747,98.371470,98.390563,98.404646,98.430988] # lon of stantion point

In [4]:
def readSpeed(time) : #calculate distance from lat and lot then find the speed by v = s/t
    

    resp = requests.get(url) #request url
    data_disc = json.loads(resp.text) #converst to json format
    #convest data to radians value for process in the next step
    lat1 = np.radians(float(data_disc["feeds"][0]["field1"])) 
    lon1 = np.radians(float(data_disc["feeds"][0]["field2"]))
    
    lat2 = np.radians(float(data_disc["feeds"][1]["field1"]))
    lon2 = np.radians(float(data_disc["feeds"][1]["field2"]))
    #find difference between point 1 and point 2
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    distance = R * c
    speed = (distance*3600)/time; #km/hr

    return speed


In [5]:
def distanceDriver_Station(latSt,lonSt):#calculate distance from location of driver and station
    
    resp = requests.get(url)
    data_disc = json.loads(resp.text)
    
    lat1 = np.radians(float(data_disc["feeds"][1]["field1"]))
    lon1 = np.radians(float(data_disc["feeds"][1]["field2"]))
    
    lat2 = np.radians(latSt)
    lon2 = np.radians(lonSt)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    
    return distance

In [6]:
def addData(dt1,dt2,dt3,dt4,dt5,dt6,dt7,dt8): # add the data into thing speak
    urlWrite = "https://api.thingspeak.com/update?api_key="+ Rkey + "&field1=" + str(dt1) + "&field2=" +  str(dt2) + "&field3=" +  str(dt3) + "&field4=" +  str(dt4) + "&field5=" +  str(dt5) + "&field6=" +  str(dt6) + "&field7=" +  str(dt7) + "&field8=" +  str(dt8)
    requests.get(urlWrite)

In [7]:
bc = blockchain.Blockchain()# create first blockchain

In [8]:
while (bc.is_chain_valid()):# check validation of blockchain
    
    disDriver_station = [distanceDriver_Station(latStation[0],lonstation[0]),
                         distanceDriver_Station(latStation[1],lonstation[1]),
                         distanceDriver_Station(latStation[2],lonstation[2]),
                         distanceDriver_Station(latStation[3],lonstation[3]),
                         distanceDriver_Station(latStation[4],lonstation[4]),
                         distanceDriver_Station(latStation[5],lonstation[5]),
                         distanceDriver_Station(latStation[6],lonstation[6])]

    station_dis = {'station 1': disDriver_station[0],
                  'station 2': disDriver_station[1],
                  'station 3': disDriver_station[2],
                  'station 4': disDriver_station[3],
                  'station 5': disDriver_station[4],
                  'station 6': disDriver_station[5],
                  'station 7': disDriver_station[6],}

    st_minDis = min(station_dis, key = lambda k: station_dis[k])# find name of the station nearby the driver
    minDis = station_dis[st_minDis]# find min distance of station nearby the driver
    speed = readSpeed(30) # calculate the speed when t  = 30s because driver divec will be update every 30 s
    estimate = (minDis/speed)* 60 # for min
    
    bc.mine_block("Station :" + str(st_minDis) + ",Distance :"+ str(minDis) + ",Speed :" + str(speed) + ",Estimate time :" + str(estimate))
    #create the data by use mine_blockmethod
    addData(st_minDis,minDis,speed,estimate,bc.is_chain_valid(),bc.chain[len(bc.chain)-1]["index"],bc.chain[len(bc.chain)-1]["proof"],bc.chain[len(bc.chain)-1]["previous_hash"])
    print(bc.chain[len(bc.chain)-1])# print the current block
    time.sleep(30)

{'index': 2, 'timestamp': '2022-09-27 00:05:04.329703', 'data': 'Station :station 5,Distance :5.802091323077895,Speed :888.0029233891671,Estimate time :0.3920319068950945', 'proof': 1808, 'previous_hash': '9631bbe5c6916992404be7bffe8bea2b3916c94cd2a23e2a689a9159826e5d10'}
{'index': 3, 'timestamp': '2022-09-27 00:05:45.017712', 'data': 'Station :station 5,Distance :5.802091323077895,Speed :888.0029233891671,Estimate time :0.3920319068950945', 'proof': 11367, 'previous_hash': '93b10e88c2a4a8addf089480f4b8f8790c1c7a8534d2919abcf52cee466ff763'}
{'index': 4, 'timestamp': '2022-09-27 00:06:26.278413', 'data': 'Station :station 5,Distance :5.802091323077895,Speed :888.0029233891671,Estimate time :0.3920319068950945', 'proof': 2171, 'previous_hash': '030c9e0cfa0ae6d08c6eaab17f13dc6169d7933239c3772fff26c4975100c693'}


KeyboardInterrupt: 